## Imports

In [7]:
import pandas as pd
import numpy as np
import re
import requests
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

load_dotenv()

AWSDBHOST = os.getenv("AWSDBHOST")
AWSDBUSER = os.getenv("AWSDBUSER")
AWSDBPASS = os.getenv("AWSDBPASS")
AWSDBNAME = os.getenv("AWSDBNAME")

## Extraction

In [8]:
engine = create_engine(f"mysql+pymysql://{AWSDBUSER}:{AWSDBPASS}@{AWSDBHOST}/{AWSDBNAME}", echo=True)

In [9]:
query = text("SELECT * FROM address ")
df_address = pd.read_sql(query, engine)

df_address

2022-05-18 22:58:01,027 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-05-18 22:58:01,028 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-18 22:58:01,052 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-05-18 22:58:01,053 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-18 22:58:01,063 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-05-18 22:58:01,064 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-18 22:58:01,104 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-05-18 22:58:01,104 INFO sqlalchemy.engine.Engine [generated in 0.00054s] {'table_schema': 'dataengineer', 'table_name': 'SELECT * FROM address '}
2022-05-18 22:58:01,127 INFO sqlalchemy.engine.Engine SELECT * FROM address 
2022-05-18 22:58:01,127 INFO sqlalchemy.engine.Engine [generated in 0.00052s] {}


,address_id,address,city,postal_code
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217
3,4,6 BD DES ETINES,LE COTEAU,42120
4,5,35 AV DU 159EME RIA,BRIANCON,5100
...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100
558,602,6 AV JEAN JAURES,FEURS,42110
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800


## Transformation

In [10]:
df_address['postal_code'] = df_address['postal_code'].astype(str).apply(lambda x: x.zfill(5))
df_address['city'] = df_address['city'].apply(lambda x: re.sub('\s\d+EME', '', x)).apply(lambda x: re.sub('\s\d+', '', x))
df_address

,address_id,address,city,postal_code
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217
3,4,6 BD DES ETINES,LE COTEAU,42120
4,5,35 AV DU 159EME RIA,BRIANCON,05100
...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100
558,602,6 AV JEAN JAURES,FEURS,42110
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,08000
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800


In [11]:
lat_dict = {}
lon_dict = {}

for i in df_address.itertuples():
    index_num = i.address_id
    street = i.address
    city = i.city
    postalcode = i.postal_code
    try:
        r = requests.get(f"https://nominatim.openstreetmap.org/search?city={city}&street={street}&postalcode={postalcode}&format=json").json()
        lat_dict[index_num] = (r[0]['lat'])        
        lon_dict[index_num] = (r[0]['lon'])
    except IndexError:
        try:
            r = requests.get(f"https://nominatim.openstreetmap.org/search?city={city}&postalcode={postalcode}&format=json").json()
            lat_dict[index_num] = (r[0]['lat'])        
            lon_dict[index_num] = (r[0]['lon'])
        except:
            r = requests.get(f"https://nominatim.openstreetmap.org/search?postalcode={postalcode}&format=json").json()
            lat_dict[index_num] = (r[0]['lat'])        
            lon_dict[index_num] = (r[0]['lon'])

df_address['latitude'] = df_address['address_id'].map(lat_dict).astype(float)
df_address['longitude'] = df_address['address_id'].map(lon_dict).astype(float)

df_address

,address_id,address,city,postal_code,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.200998,5.605799
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.290905,4.045789
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.361340,-4.766800
3,4,6 BD DES ETINES,LE COTEAU,42120,46.021659,4.092120
4,5,35 AV DU 159EME RIA,BRIANCON,05100,44.898404,6.643631
...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,48.868459,2.468415
558,602,6 AV JEAN JAURES,FEURS,42110,45.743563,4.226839
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,08000,49.773546,4.708829
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,48.953836,2.292919


## Loading

In [12]:
query = "ALTER TABLE address ADD latitude FLOAT, ADD longitude FLOAT;"
with engine.connect() as con:
    con.execute(query)

2022-05-18 23:03:17,790 INFO sqlalchemy.engine.Engine ALTER TABLE address ADD latitude FLOAT, ADD longitude FLOAT;
2022-05-18 23:03:17,793 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-18 23:03:17,833 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
for i in lat_dict.keys():
    query = f"UPDATE address SET latitude = {lat_dict.get(i)}, longitude = {lon_dict.get(i)} WHERE address_id = {i}"
    with engine.connect() as con:
        con.execute(query)

2022-05-18 23:03:17,885 INFO sqlalchemy.engine.Engine UPDATE address SET latitude = 43.2009979, longitude = 5.6057988 WHERE address_id = 1
2022-05-18 23:03:17,886 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-18 23:03:17,898 INFO sqlalchemy.engine.Engine COMMIT
2022-05-18 23:03:17,921 INFO sqlalchemy.engine.Engine UPDATE address SET latitude = 48.2909051, longitude = 4.0457894 WHERE address_id = 2
2022-05-18 23:03:17,921 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-18 23:03:17,942 INFO sqlalchemy.engine.Engine COMMIT
2022-05-18 23:03:17,970 INFO sqlalchemy.engine.Engine UPDATE address SET latitude = 48.3613399, longitude = -4.7667997 WHERE address_id = 3
2022-05-18 23:03:17,971 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-05-18 23:03:17,982 INFO sqlalchemy.engine.Engine COMMIT
2022-05-18 23:03:18,010 INFO sqlalchemy.engine.Engine UPDATE address SET latitude = 46.021659, longitude = 4.0921204 WHERE address_id = 4
2022-05-18 23:03:18,010 INFO sqlalchemy.engine.Engine [raw s

In [14]:
df_final = pd.read_sql("SELECT * FROM address", engine)
df_final

2022-05-18 23:03:39,376 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-05-18 23:03:39,377 INFO sqlalchemy.engine.Engine [cached since 338.3s ago] {'table_schema': 'dataengineer', 'table_name': 'SELECT * FROM address'}
2022-05-18 23:03:39,400 INFO sqlalchemy.engine.Engine SELECT * FROM address
2022-05-18 23:03:39,400 INFO sqlalchemy.engine.Engine [raw sql] {}


,address_id,address,city,postal_code,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.2010,5.60580
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.2909,4.04579
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.3613,-4.76680
3,4,6 BD DES ETINES,LE COTEAU,42120,46.0217,4.09212
4,5,35 AV DU 159EME RIA,BRIANCON,5100,44.8984,6.64363
...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,48.8685,2.46842
558,602,6 AV JEAN JAURES,FEURS,42110,45.7436,4.22684
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,49.7735,4.70883
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,48.9538,2.29292


## SQL Query

In [16]:
query = """
SELECT c.last_name, c.first_name, a.address, a.city, a.postal_code, a.latitude, a.longitude, COUNT(r.rental_id) AS total_rentals
FROM rental r
JOIN customer c on c.customer_id=r.customer_id
JOIN address a on a.address_id=c.address_id
GROUP BY r.customer_id
ORDER BY total_rentals DESC
LIMIT 1
"""

df_bernardo = pd.read_sql(query, engine)
df_bernardo


2022-05-18 23:24:55,133 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-05-18 23:24:55,134 INFO sqlalchemy.engine.Engine [cached since 1614s ago] {'table_schema': 'dataengineer', 'table_name': '\nSELECT c.last_name, c.first_name, a.address, a.city, a.postal_code, a.latitude, a.longitude, COUNT(r.rental_id) AS total_rentals\nFROM rental r\nJO ... (4 characters truncated) ... ustomer c on c.customer_id=r.customer_id\nJOIN address a on a.address_id=c.address_id\nGROUP BY r.customer_id\nORDER BY total_rentals DESC\nLIMIT 1\n'}
2022-05-18 23:24:55,157 INFO sqlalchemy.engine.Engine 
SELECT c.last_name, c.first_name, a.address, a.city, a.postal_code, a.latitude, a.longitude, COUNT(r.rental_id) AS total_rentals
FROM rental r
JOIN customer c on c.customer_id=r.customer_id
JOIN address a on a.address_id=c.address_id
GROUP BY r.customer_id
ORDER BY total_rentals DESC
LIMIT 1

2022-05-18 23:24:5

,last_name,first_name,address,city,postal_code,latitude,longitude,total_rentals
0,HUNT,ELEANOR,17 IMP DES JARDINS,VALLEIRY,74520,46.1068,5.96634,46
